In [1]:
import json
import pandas as pd
import random
from datetime import datetime, timedelta

In [ ]:
# UPDATE_PARTICIPANT_TO_ACTION,    // ACTION 참여자 수정(참여자 추가 및 삭제) 이벤트
# CREATE_PROJECT_PROGRESS_ACTION,  // ACTION 생성 이벤트
# DELETE_PROJECT_PROGRESS_ACTION,  // ACTION 삭제 이벤트
# DONE_PROJECT_PROGRESS_ACTION     // ACTION 상태 완료로 변경이벤트

In [51]:
LOG_DATA_PATH = "data\\user-actions.json"

# 중첩 구조 평탄화해서 읽기
with open(LOG_DATA_PATH, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f if line.strip()]
df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   event                 5 non-null      object
 1   userId                5 non-null      object
 2   username              5 non-null      object
 3   timestamp             5 non-null      object
 4   workspaceId           5 non-null      int64 
 5   details.participants  5 non-null      object
 6   details.state         5 non-null      object
 7   details.name          5 non-null      object
 8   details.importance    5 non-null      int64 
 9   details.startDate     1 non-null      object
 10  details.endDate       1 non-null      object
dtypes: int64(2), object(9)
memory usage: 568.0+ bytes


In [52]:
df

,event,userId,username,timestamp,workspaceId,details.participants,details.state,details.name,details.importance,details.startDate,details.endDate
0,CREATE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-17T04:50:09.778981259Z,1,"[{'username': 'testuser1', 'userId': 1}]",IN_PROGRESS,회원 프론트 화면 피그마 디자인,3,NaN,NaN
1,DONE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-17T04:52:40.68515608Z,1,"[{'userId': 1, 'username': 'testuser1'}]",DONE,회원 프론트 화면 피그마 디자인,3,2025-06-10T09:00:00,2025-06-17T04:52:40.668571836
2,CREATE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-17T04:55:44.921068578Z,1,"[{'userId': 1, 'username': 'testuser1'}]",IN_PROGRESS,회원 프론트 화면 프론트 설계,3,NaN,NaN
3,UPDATE_PARTICIPANT_TO_ACTION,1,testuser1,2025-06-17T04:56:30.969938025Z,1,"[{'userId': 1, 'username': 'testuser1'}, {'use...",IN_PROGRESS,회원 프론트 화면 프론트 설계,3,NaN,NaN
4,DELETE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-17T04:56:53.084405091Z,1,"[{'userId': 1, 'username': 'testuser1'}]",IN_PROGRESS,회원 프론트 화면 프론트 설계,3,NaN,NaN


In [50]:
print("df['event'].unique(): ", df['event'].unique())
print("df['userId'].unique(): ", df['userId'].unique())
print("df['workspaceId'].unique(): ", df['workspaceId'].unique())
print("df['details.importance'].unique(): ", df['details.importance'].unique())
print("df['details.state'].unique(): ", df['details.state'].unique())

df['event'].unique():  ['CREATE_PROJECT_PROGRESS_ACTION' 'DONE_PROJECT_PROGRESS_ACTION'
 'UPDATE_PARTICIPANT_TO_ACTION' 'DELETE_PROJECT_PROGRESS_ACTION']
df['userId'].unique():  ['1']
df['workspaceId'].unique():  [1]
df['details.importance'].unique():  [3]
df['details.state'].unique():  ['IN_PROGRESS' 'DONE']


In [31]:
df_user1 = df[df['userId'] == 1]
df_user2 = df[df['userId'] == 2]
df_user3 = df[df['userId'] == 3]
df_user4 = df[df['userId'] == 4]
df_user5 = df[df['userId'] == 5]

In [ ]:
# 1. 담당자 불균형 분석 -> Bar plot
# - point columns = details.participants, details.importance
# - 작업이 특정 개발자에게 몰리는가 -> userId별 분류 -> status별 분류 -> 시각화
#   - status는 왜? 이미 진행 중인 작업이 많은 개발자에게 얼마나 더 할당되어 있는가
# - 각 개발자가 맡은 업무의 중요도 분포는 어떻게 되는가
# - status는 중요하지 않음, participants 체크가 중요

In [ ]:
# 2. 중요도에 따른 평균 작업 처리 기간(시간) → Line chart
# - userId별 분류 -> importance별 분류
# - status=="DONE"인 로그의 timestamp.. INPROGRESS에서 DONE으로 바뀐 시간 간격 계산이 되나?
# - 